In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


mb_size = 32
X_dim = 784
z_dim = 10
h_dim = 128

mnist = input_data.read_data_sets('/tmp/MNIST_data', one_hot=True)


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


X = tf.placeholder(tf.float32, shape=[None, X_dim])

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


z = tf.placeholder(tf.float32, shape=[None, z_dim])

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    out = tf.matmul(D_h1, D_W2) + D_b2
    return out


G_sample = generator(z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

D_loss = tf.reduce_mean(D_real) - tf.reduce_mean(D_fake)
G_loss = -tf.reduce_mean(D_fake)

D_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(-D_loss, var_list=theta_D))
G_solver = (tf.train.RMSPropOptimizer(learning_rate=1e-4)
            .minimize(G_loss, var_list=theta_G))

clip_D = [p.assign(tf.clip_by_value(p, -0.01, 0.01)) for p in theta_D]

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(1000000):
    for _ in range(5):
        X_mb, _ = mnist.train.next_batch(mb_size)

        _, D_loss_curr, _ = sess.run(
            [D_solver, D_loss, clip_D],
            feed_dict={X: X_mb, z: sample_z(mb_size, z_dim)}
        )

    _, G_loss_curr = sess.run(
        [G_solver, G_loss],
        feed_dict={z: sample_z(mb_size, z_dim)}
    )

    if it % 100 == 0:
        print('Iter: {}; D loss: {:.4}; G_loss: {:.4}'
              .format(it, D_loss_curr, G_loss_curr))

        if it % 1000 == 0:
            samples = sess.run(G_sample, feed_dict={z: sample_z(16, z_dim)})

            fig = plot(samples)
            plt.savefig('out/{}.png'
                        .format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)

Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Iter: 0; D loss: -0.009449; G_loss: -0.0132
Iter: 100; D loss: 1.89; G_loss: 1.552
Iter: 200; D loss: 1.761; G_loss: 1.379
Iter: 300; D

Iter: 13000; D loss: 0.03365; G_loss: -0.01012
Iter: 13100; D loss: 0.03204; G_loss: -0.01144
Iter: 13200; D loss: 0.03315; G_loss: -0.009275
Iter: 13300; D loss: 0.03243; G_loss: -0.01075
Iter: 13400; D loss: 0.02972; G_loss: -0.009323
Iter: 13500; D loss: 0.03135; G_loss: -0.009764
Iter: 13600; D loss: 0.03103; G_loss: -0.01098
Iter: 13700; D loss: 0.02558; G_loss: -0.01115
Iter: 13800; D loss: 0.03056; G_loss: -0.01046
Iter: 13900; D loss: 0.03057; G_loss: -0.009376
Iter: 14000; D loss: 0.0305; G_loss: -0.01128
Iter: 14100; D loss: 0.02812; G_loss: -0.01057
Iter: 14200; D loss: 0.02656; G_loss: -0.0137
Iter: 14300; D loss: 0.0262; G_loss: -0.01364
Iter: 14400; D loss: 0.02539; G_loss: -0.01054
Iter: 14500; D loss: 0.02638; G_loss: -0.0125
Iter: 14600; D loss: 0.02954; G_loss: -0.01522
Iter: 14700; D loss: 0.02527; G_loss: -0.01139
Iter: 14800; D loss: 0.02624; G_loss: -0.01223
Iter: 14900; D loss: 0.02323; G_loss: -0.008997
Iter: 15000; D loss: 0.02766; G_loss: -0.01134
Iter: 15100;

Iter: 30400; D loss: 0.02239; G_loss: -0.006142
Iter: 30500; D loss: 0.01653; G_loss: -0.00929
Iter: 30600; D loss: 0.02339; G_loss: -0.008591
Iter: 30700; D loss: 0.01736; G_loss: -0.007159
Iter: 30800; D loss: 0.0183; G_loss: -0.01074
Iter: 30900; D loss: 0.01563; G_loss: -0.0125
Iter: 31000; D loss: 0.01963; G_loss: -0.01141
Iter: 31100; D loss: 0.0175; G_loss: -0.00862
Iter: 31200; D loss: 0.02248; G_loss: -0.01278
Iter: 31300; D loss: 0.01393; G_loss: -0.008673
Iter: 31400; D loss: 0.0205; G_loss: -0.008006
Iter: 31500; D loss: 0.01644; G_loss: -0.01425
Iter: 31600; D loss: 0.01774; G_loss: -0.008349
Iter: 31700; D loss: 0.01888; G_loss: -0.0122
Iter: 31800; D loss: 0.01988; G_loss: -0.01083
Iter: 31900; D loss: 0.02002; G_loss: -0.01183
Iter: 32000; D loss: 0.01882; G_loss: -0.01143
Iter: 32100; D loss: 0.02044; G_loss: -0.007591
Iter: 32200; D loss: 0.01768; G_loss: -0.01218
Iter: 32300; D loss: 0.01631; G_loss: -0.007548
Iter: 32400; D loss: 0.01781; G_loss: -0.01023
Iter: 3250

Iter: 47900; D loss: 0.01359; G_loss: -0.01402
Iter: 48000; D loss: 0.01661; G_loss: -0.01191
Iter: 48100; D loss: 0.01406; G_loss: -0.008066
Iter: 48200; D loss: 0.01601; G_loss: -0.006647
Iter: 48300; D loss: 0.01174; G_loss: -0.007669
Iter: 48400; D loss: 0.01243; G_loss: -0.01262
Iter: 48500; D loss: 0.01319; G_loss: -0.01172
Iter: 48600; D loss: 0.01439; G_loss: -0.01353
Iter: 48700; D loss: 0.01316; G_loss: -0.01684
Iter: 48800; D loss: 0.01124; G_loss: -0.01608
Iter: 48900; D loss: 0.01517; G_loss: -0.01326
Iter: 49000; D loss: 0.01436; G_loss: -0.01273
Iter: 49100; D loss: 0.01793; G_loss: -0.01331
Iter: 49200; D loss: 0.01607; G_loss: -0.01378
Iter: 49300; D loss: 0.01489; G_loss: -0.01788
Iter: 49400; D loss: 0.01638; G_loss: -0.0165
Iter: 49500; D loss: 0.01781; G_loss: -0.009009
Iter: 49600; D loss: 0.01359; G_loss: -0.01322
Iter: 49700; D loss: 0.01631; G_loss: -0.01276
Iter: 49800; D loss: 0.01255; G_loss: -0.01127
Iter: 49900; D loss: 0.01599; G_loss: -0.01535
Iter: 5000

Iter: 65400; D loss: 0.01263; G_loss: -0.01196
Iter: 65500; D loss: 0.0134; G_loss: -0.012
Iter: 65600; D loss: 0.01146; G_loss: -0.0136
Iter: 65700; D loss: 0.01218; G_loss: -0.01151
Iter: 65800; D loss: 0.01075; G_loss: -0.01259
Iter: 65900; D loss: 0.01381; G_loss: -0.0138
Iter: 66000; D loss: 0.01085; G_loss: -0.01219
Iter: 66100; D loss: 0.01201; G_loss: -0.01214
Iter: 66200; D loss: 0.01423; G_loss: -0.01337
Iter: 66300; D loss: 0.012; G_loss: -0.01482
Iter: 66400; D loss: 0.01542; G_loss: -0.01755
Iter: 66500; D loss: 0.01476; G_loss: -0.01146
Iter: 66600; D loss: 0.01169; G_loss: -0.009098
Iter: 66700; D loss: 0.01373; G_loss: -0.01513
Iter: 66800; D loss: 0.01731; G_loss: -0.01296
Iter: 66900; D loss: 0.01282; G_loss: -0.01516
Iter: 67000; D loss: 0.01201; G_loss: -0.01681
Iter: 67100; D loss: 0.0122; G_loss: -0.01403
Iter: 67200; D loss: 0.0155; G_loss: -0.01508
Iter: 67300; D loss: 0.01249; G_loss: -0.01741
Iter: 67400; D loss: 0.01319; G_loss: -0.01119
Iter: 67500; D loss: 

...

Iter: 502200; D loss: 0.007054; G_loss: -0.01177
Iter: 502300; D loss: 0.00623; G_loss: -0.005913
Iter: 502400; D loss: 0.002735; G_loss: -0.01277
Iter: 502500; D loss: 0.006389; G_loss: -0.01028
Iter: 502600; D loss: 0.005281; G_loss: -0.006939
Iter: 502700; D loss: 0.004162; G_loss: -0.01073
Iter: 502800; D loss: 0.005037; G_loss: -0.008323
Iter: 502900; D loss: 0.006218; G_loss: -0.002649
Iter: 503000; D loss: 0.005087; G_loss: -0.0106
Iter: 503100; D loss: 0.006914; G_loss: -0.01195
Iter: 503200; D loss: 0.008166; G_loss: -0.008495
Iter: 503300; D loss: 0.006529; G_loss: -0.002894
Iter: 503400; D loss: 0.007305; G_loss: -0.01406
Iter: 503500; D loss: 0.004984; G_loss: -0.009491
Iter: 503600; D loss: 0.003676; G_loss: -0.006425
Iter: 503700; D loss: 0.004013; G_loss: -0.006789
Iter: 503800; D loss: 0.006478; G_loss: -0.005571
Iter: 503900; D loss: 0.003909; G_loss: -0.00976
Iter: 504000; D loss: 0.001815; G_loss: -0.01209
Iter: 504100; D loss: 0.007642; G_loss: -0.01492
Iter: 504200

Iter: 518800; D loss: 0.004631; G_loss: -0.01236
Iter: 518900; D loss: 0.007604; G_loss: -0.00425
Iter: 519000; D loss: 0.005601; G_loss: -0.005452
Iter: 519100; D loss: 0.008752; G_loss: -0.00922
Iter: 519200; D loss: 0.005122; G_loss: -0.003206
Iter: 519300; D loss: 0.005107; G_loss: -0.01359
Iter: 519400; D loss: 0.005533; G_loss: -0.004889
Iter: 519500; D loss: 0.00715; G_loss: -0.007954
Iter: 519600; D loss: 0.001938; G_loss: -0.007729
Iter: 519700; D loss: 0.00584; G_loss: -0.01323
Iter: 519800; D loss: 0.003365; G_loss: -0.01454
Iter: 519900; D loss: 0.006677; G_loss: -0.0122
Iter: 520000; D loss: 0.006561; G_loss: -0.01114
Iter: 520100; D loss: 0.00364; G_loss: -0.01383
Iter: 520200; D loss: 0.005029; G_loss: -0.01034
Iter: 520300; D loss: 0.004623; G_loss: -0.0124
Iter: 520400; D loss: 0.005295; G_loss: -0.008034
Iter: 520500; D loss: 0.003668; G_loss: -0.01097
Iter: 520600; D loss: 0.00515; G_loss: -0.01238
Iter: 520700; D loss: 0.00416; G_loss: -0.01044
Iter: 520800; D loss:

Iter: 535400; D loss: 0.006756; G_loss: -0.00939
Iter: 535500; D loss: 0.007452; G_loss: -0.001639
Iter: 535600; D loss: 0.00485; G_loss: -0.008964
Iter: 535700; D loss: 0.007925; G_loss: -0.0158
Iter: 535800; D loss: 0.003388; G_loss: -0.01145
Iter: 535900; D loss: 0.004109; G_loss: -0.01642
Iter: 536000; D loss: -0.0003192; G_loss: -0.009961
Iter: 536100; D loss: 0.001825; G_loss: -0.00879
Iter: 536200; D loss: 0.004206; G_loss: -0.003282
Iter: 536300; D loss: 0.006335; G_loss: -0.008622
Iter: 536400; D loss: 0.005186; G_loss: -0.007034
Iter: 536500; D loss: 0.004205; G_loss: -0.007497
Iter: 536600; D loss: 0.004837; G_loss: -0.00622
Iter: 536700; D loss: 0.003653; G_loss: -0.01504
Iter: 536800; D loss: 0.008007; G_loss: -0.0119
Iter: 536900; D loss: 0.006707; G_loss: -0.01124
Iter: 537000; D loss: 0.005288; G_loss: -0.007102
Iter: 537100; D loss: 0.006815; G_loss: -0.00783
Iter: 537200; D loss: 0.00424; G_loss: -0.01008
Iter: 537300; D loss: 0.007069; G_loss: -0.008922
Iter: 537400;

Iter: 552000; D loss: 0.004501; G_loss: -0.009123
Iter: 552100; D loss: 0.006871; G_loss: -0.0004834
Iter: 552200; D loss: 0.005231; G_loss: -0.009836
Iter: 552300; D loss: 0.003542; G_loss: -0.0127
Iter: 552400; D loss: 0.001135; G_loss: -0.01141
Iter: 552500; D loss: 0.007117; G_loss: -0.01105
Iter: 552600; D loss: 0.004566; G_loss: -0.0056
Iter: 552700; D loss: 0.004047; G_loss: -0.01833
Iter: 552800; D loss: 0.005481; G_loss: -0.009545
Iter: 552900; D loss: 0.003665; G_loss: -0.01166
Iter: 553000; D loss: 0.006963; G_loss: 0.0003503
Iter: 553100; D loss: 0.006207; G_loss: -0.01109
Iter: 553200; D loss: 0.003407; G_loss: -0.01494
Iter: 553300; D loss: 0.001315; G_loss: -0.009535
Iter: 553400; D loss: 0.007837; G_loss: -0.01021
Iter: 553500; D loss: 0.002869; G_loss: -0.0107
Iter: 553600; D loss: 0.006977; G_loss: -0.0198
Iter: 553700; D loss: 0.002185; G_loss: -0.002403
Iter: 553800; D loss: 0.005585; G_loss: -0.009458
Iter: 553900; D loss: 0.001019; G_loss: -0.0116
Iter: 554000; D 

Iter: 568600; D loss: 0.001825; G_loss: -0.009931
Iter: 568700; D loss: 0.005564; G_loss: -0.005898
Iter: 568800; D loss: 0.004435; G_loss: -0.00805
Iter: 568900; D loss: 0.0057; G_loss: -0.003662
Iter: 569000; D loss: 0.004032; G_loss: -0.007553
Iter: 569100; D loss: 0.007451; G_loss: -0.002509
Iter: 569200; D loss: 0.005401; G_loss: -0.004752
